# تحليل بيانات مكتبة الأطفال 📚
هذا الدفتر يقدّم تحليلًا وصفيًا وتنبؤيًا لبيانات استعارة الكتب والغرامات في مكتبة الأطفال، مع رسوم بيانية وعناوين عربية.
الملف المستخدم: `library_children_sample_en.xlsx`.

## 1) تحميل البيانات ومعاينتها

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
path = r'library_children_sample_en.xlsx'
df = pd.read_excel(path)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include=[np.number]).T

## 2) تحديد أنواع الأعمدة

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object','category','bool']).columns.tolist()
numeric_cols, categorical_cols

## 3) رسوم بيانية أساسية

In [ ]:
# هيستوجرام للأعمار (إن وُجدت)
col_age = None
for c in df.columns:
    if 'age' in c.lower():
        col_age = c; break
if col_age is not None:
    df[col_age].dropna().plot(kind='hist', bins=10)
    plt.title('توزيع الأعمار')
    plt.xlabel('العمر')
    plt.ylabel('التكرار')
    plt.show()

In [ ]:
# أكثر الأنواع قراءةً (إن وُجدت)
col_genre = None
for c in df.columns:
    if 'genre' in c.lower():
        col_genre = c; break
if col_genre is not None:
    df[col_genre].value_counts().head(10).plot(kind='bar')
    plt.title('أكثر الأنواع قراءةً (أعلى 10)')
    plt.xlabel('النوع')
    plt.ylabel('عدد الاستعارات')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout(); plt.show()

In [ ]:
# Boxplot للغرامات حسب الجنس/المستوى القرائي إن وُجدت الأعمدة
col_fine = None
for c in df.columns:
    if 'fine' in c.lower():
        col_fine = c; break
group_col = None
for c in df.columns:
    if c.lower() in ['gender','reading_level']:
        group_col = c; break
if col_fine is not None and group_col is not None:
    groups = [g[col_fine].dropna().values for _, g in df.groupby(group_col)]
    labels = [str(k) for k,_ in df.groupby(group_col)]
    if len(groups) >= 2:
        plt.boxplot(groups, labels=labels)
        plt.title(f'الغرامات بحسب {group_col}')
        plt.xlabel(group_col)
        plt.ylabel('قيمة الغرامة')
        plt.show()

In [ ]:
# علاقة عدد الكتب المستعارة بالغرامات (إن توفّر العمودان)
col_borrow = None
for c in df.columns:
    if 'borrowed_books' in c.lower():
        col_borrow = c; break
if col_borrow is not None and col_fine is not None:
    plt.scatter(df[col_borrow], df[col_fine])
    plt.title('العلاقة بين عدد الكتب المستعارة والغرامات')
    plt.xlabel('عدد الكتب المستعارة')
    plt.ylabel('قيمة الغرامة')
    plt.show()

## 4) مصفوفة الارتباط للمتغيرات الرقمية

In [ ]:
nums = df.select_dtypes(include=[np.number])
if nums.shape[1] >= 2:
    corr = nums.corr(numeric_only=True)
    im = plt.imshow(corr, interpolation='nearest')
    plt.title('مصفوفة الارتباط (رقمي)')
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha='right')
    plt.yticks(range(len(corr.index)), corr.index)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout(); plt.show()
corr

## 5) نموذج تصنيف: التنبؤ بالتأخر في الإرجاع `Return_Status`

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# تجهيز الهدف
col_return = None
for c in df.columns:
    if 'return_status' in c.lower():
        col_return = c; break
if col_return is None:
    print('⚠️ لم أجد عمود Return_Status، سيتم تجاوز هذا القسم.')
else:
    y = (df[col_return].astype(str).str.strip().str.lower().isin(['late','yes','true','1'])).astype(int)
    X = df.drop(columns=[col_return]).copy()
    X = pd.get_dummies(X, drop_first=True)
    X = X.select_dtypes(include=[np.number]).fillna(0)
    if X.shape[0] >= 8 and X.shape[1] > 0 and y.nunique()==2:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
        clf = LogisticRegression(max_iter=200).fit(X_train, y_train)
        preds = clf.predict(X_test)
        acc = accuracy_score(y_test, preds)
        prec = precision_score(y_test, preds, zero_division=0)
        rec = recall_score(y_test, preds, zero_division=0)
        f1 = f1_score(y_test, preds, zero_division=0)
        print('الدقة:', round(acc,3),'، الدقة النوعية:', round(prec,3),'، الاستدعاء:', round(rec,3),'، F1:', round(f1,3))
        print('مصفوفة الالتباس:\n', confusion_matrix(y_test, preds))
    else:
        print('⚠️ بيانات غير كافية/غير ثنائية للتصنيف.')

## 6) نموذج انحدار: التنبؤ بعدد الكتب المستعارة `Borrowed_Books`

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

col_borrow = None
for c in df.columns:
    if 'borrowed_books' in c.lower():
        col_borrow = c; break
if col_borrow is None:
    print('⚠️ لم أجد عمود Borrowed_Books، سيتم تجاوز هذا القسم.')
else:
    y = df[col_borrow].astype(float)
    X = df.drop(columns=[col_borrow]).copy()
    X = pd.get_dummies(X, drop_first=True)
    X = X.select_dtypes(include=[np.number]).fillna(0)
    valid = y.notna()
    X, y = X.loc[valid], y.loc[valid]
    if X.shape[0] >= 8 and X.shape[1] > 0:
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
        lr = LinearRegression().fit(X_train, y_train)
        preds = lr.predict(X_test)
        r2 = r2_score(y_test, preds)
        mae = mean_absolute_error(y_test, preds)
        rmse = mean_squared_error(y_test, preds, squared=False)
        print('R²:', round(r2,3),' MAE:', round(mae,3),' RMSE:', round(rmse,3))
    else:
        print('⚠️ بيانات غير كافية للانحدار.')